## Calibration

The `Calibration` class provides a way to adjust weights of observations in a dataset to match specified target values. This is commonly used in survey research and policy modeling for rebalancing datasets to better represent desired population characteristics. 

The calibration process uses an optimization algorithm to find weights that minimize the distance from the original weights while achieving the target constraints.

## Basic usage

### Parameters

`__init__(data, weights, targets)`

- `data` (pd.DataFrame): The dataset to be calibrated. This should contain all the variables you want to use for calibration.
- `weights` (np.ndarray): Initial weights for each observation in the dataset. Typically starts as an array of ones for equal weighting.
- `targets` (np.ndarray): Target values that the calibration process should achieve. These correspond to the desired weighted sums.

Calibration can be easily done by initializing the `Calibration` class, passing in the parameters above. Then `calibrate()` method performs the actual calibration using the reweight function. This method:
- Adjusts the weights to better match the target values
- May subsample the data for efficiency
- Updates both `self.weights` and `self.data` with the calibrated results

## Example

Below is a complete example showing how to calibrate a dataset to match income targets for specific age groups:

In [9]:
from microcalibrate.calibration import Calibration
import logging
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

logging.basicConfig(
    level=logging.INFO,
)

# Create a sample dataset with age and income data
random_generator = np.random.default_rng(0)
data = pd.DataFrame({
    "age": np.append(random_generator.integers(18, 70, size=120), 71), 
    "income": random_generator.normal(40000, 10000, size=121),
})

# Set initial weights (all one in this example)
weights = np.ones(len(data))

# Calculate target values: total income for age groups 20-30 and 40-50 (as an example) or employ existing targets
targets_matrix = pd.DataFrame({
    "income_aged_20_30": ((data["age"] >= 20) & (data["age"] <= 30)).astype(float) * data["income"],
    "income_aged_40_50": ((data["age"] >= 40) & (data["age"] <= 50)).astype(float) * data["income"],
    "income_aged_71" : (data["age"] == 71).astype(float) * data["income"],
})

# 15% higher than the sum of data with the original weights
targets = np.array([
    (targets_matrix["income_aged_20_30"] * weights * 1000).sum(), 
    (targets_matrix["income_aged_40_50"] * weights * 1.15).sum(), 
    (targets_matrix["income_aged_71"] * weights * 1.15).sum()
])

print(f"Original weights: {weights}")
print(f"Original targets: {targets}")

Original weights: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
Original targets: [7.37032429e+08 9.76779350e+05 4.36479914e+04]


In [10]:
# Initialize the Calibration object
calibrator = Calibration(
    loss_matrix=targets_matrix,
    weights=weights, 
    targets=targets,
    noise_level=0.05,
    epochs=528,
    learning_rate=0.01,
    dropout_rate=0,
    subsample_every=0,
)

# Perform the calibration
performance_df = calibrator.calibrate()

print(f"Original dataset size: {len(targets_matrix)}")
print(f"Calibrated dataset size: {len(calibrator.loss_matrix)}")
print(f"Number of calibrated weights: {len(calibrator.weights)}")

INFO:microcalibrate.calibration:Performing basic target assessment...
INFO:microcalibrate.reweight:Starting calibration process for targets ['income_aged_20_30' 'income_aged_40_50' 'income_aged_71']: [7.37032429e+08 9.76779350e+05 4.36479914e+04]
INFO:microcalibrate.reweight:Original weights - mean: 1.0000, std: 0.0000
INFO:microcalibrate.reweight:Initial weights after noise - mean: 1.0253, std: 0.0139
Reweighting progress:   0%|          | 0/528 [00:00<?, ?epoch/s, loss=0.341, count_observations=121, weights_mean=1.03, weights_std=0.0139, weights_min=1]INFO:microcalibrate.reweight:Within 10% from targets: 0.00% 

Reweighting progress:   0%|          | 1/528 [00:00<01:16,  6.88epoch/s, loss=0.333, count_observations=121, weights_mean=1.06, weights_std=0.0507, weights_min=1]INFO:microcalibrate.reweight:Within 10% from targets: 66.67% 

INFO:microcalibrate.reweight:Epoch   10: Loss = 0.332883, Change = 0.007823 (improving)
Reweighting progress:   0%|          | 1/528 [00:00<01:16,  6.88e

Original dataset size: 121
Calibrated dataset size: 121
Number of calibrated weights: 121


In [11]:
# Calculate final weighted totals
final_totals = targets_matrix.mul(calibrator.weights, axis=0).sum().values

print(f"Target totals: {targets}")
print(f"Final calibrated totals: {final_totals}")
print(f"Difference: {final_totals - targets}")
print(f"Relative error: {(final_totals - targets) / targets * 100}")

Target totals: [7.37032429e+08 9.76779350e+05 4.36479914e+04]
Final calibrated totals: [7.37025166e+08 9.76778368e+05 4.36469951e+04]
Difference: [-7.26296939e+03 -9.82026825e-01 -9.96308503e-01]
Relative error: [-0.00098543 -0.00010054 -0.0022826 ]


In [12]:
np.testing.assert_allclose(
        final_totals,
        targets,
        rtol=0.01,  # relative tolerance
        err_msg="Calibrated totals do not match target values",
    )

In [5]:
performance_df.head()

,epoch,loss,pct_close,target_idx,target_name,target,estimate,error,abs_error,rel_abs_error
0,0,0.340706,0.000000,0,income_aged_20_30,7.370324e+08,753877.625000,-7.362786e+08,7.362786e+08,0.998977
1,0,0.340706,0.000000,1,income_aged_40_50,9.767794e+05,871178.000000,-1.056014e+05,1.056014e+05,0.108112
2,0,0.340706,0.000000,2,income_aged_71,4.364799e+04,38772.019531,-4.875973e+03,4.875973e+03,0.111711
3,10,0.332883,0.666667,0,income_aged_20_30,7.370324e+08,833397.750000,-7.361991e+08,7.361991e+08,0.998869
4,10,0.332883,0.666667,1,income_aged_40_50,9.767794e+05,957796.625000,-1.898275e+04,1.898275e+04,0.019434


In [ ]:
g20 = performance_df.query("target_name == 'income_aged_20_30'")
g40 = performance_df.query("target_name == 'income_aged_40_50'")

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "Estimate vs target: income_aged_20_30",
        "Estimate vs target: income_aged_40_50",
        "Relative absolute error: income_aged_20_30",
        "Relative absolute error: income_aged_40_50",
    ],
    shared_xaxes=True,
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
)

fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["target"],
        mode="lines", name="Target 20-30",
        line=dict(dash="dot", color="red"),
    ),
    row=1, col=1,
)
fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["estimate"],
        mode="lines", name="Estimate 20-30",
        line=dict(color="blue"),
    ),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["target"],
        mode="lines", name="Target 40-50",
        line=dict(dash="dot", color="red"),
    ),
    row=1, col=2,
)
fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["estimate"],
        mode="lines", name="Estimate 40-50",
        line=dict(color="green"),
    ),
    row=1, col=2,
)

fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["rel_abs_error"],
        mode="lines", showlegend=False,
        line=dict(color="blue"),
    ),
    row=2, col=1,
)

fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["rel_abs_error"],
        mode="lines", showlegend=False,
        line=dict(color="green"),
    ),
    row=2, col=2,
)

fig.update_layout(
    height=800, width=1050,
    title_text="Calibration performance over epochs",
    legend=dict(x=1.05, y=1, xanchor="left", yanchor="top"),
    margin=dict(r=200),
)
fig.update_xaxes(title_text="Epoch", row=2, col=1)
fig.update_xaxes(title_text="Epoch", row=2, col=2)
fig.update_yaxes(title_text="Income ($)", row=1, col=1)
fig.update_yaxes(title_text="Income ($)", row=1, col=2)
fig.update_yaxes(title_text="Relative absolute error", row=2, col=1)
fig.update_yaxes(title_text="Relative absolute error", row=2, col=2)

fig.show()